In [2]:
import os
%pwd

'/Users/brunoferreira/Downloads/End-to-End MLOps/textsummarizer/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/brunoferreira/Downloads/End-to-End MLOps/textsummarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int


In [6]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config=self.config.model_trainer
        params=self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config=ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.evaluation_strategy,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )
        return model_trainer_config



In [8]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
import torch
from datasets import load_from_disk

/Users/brunoferreira/Downloads/End-to-End MLOps/textsummarizer/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Cell 2 — ModelTrainer (versão corrigida)
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
import torch
from datasets import load_from_disk
import os

class ModelTrainer:
    def __init__(self, config):
        self.config = config

    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"


        tokenizer = AutoTokenizer.from_pretrained(
            self.config.model_ckpt,
            force_download=True
        )

        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(
            self.config.model_ckpt,
            force_download=True
        ).to(device)

        seq2seq_data_collator = DataCollatorForSeq2Seq(
            tokenizer=tokenizer,
            model=model_pegasus
        )

        # Load transformed dataset
        dataset_samsum_pt = load_from_disk(str(self.config.data_path))

        trainer_args = TrainingArguments(
            output_dir=str(self.config.root_dir),
            num_train_epochs=1,
            warmup_steps=100,
            per_device_train_batch_size=1,
            per_device_eval_batch_size=1,
            weight_decay=0.01,
            logging_steps=10,
            eval_strategy="steps",
            eval_steps=500,
            save_steps=1e6,
            gradient_accumulation_steps=16
        )

        trainer = Trainer(
            model=model_pegasus,
            args=trainer_args,
            processing_class=tokenizer,
            data_collator=seq2seq_data_collator,
            train_dataset=dataset_samsum_pt["test"],
            eval_dataset=dataset_samsum_pt["validation"]
        )

        trainer.train()

        model_pegasus.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))





In [10]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

/Users/brunoferreira/Downloads/End-to-End MLOps/textsummarizer/.venv/bin/pip: line 2: /Users/brunoferreira/Downloads/End-to-End MLOps Bootcamp/textsummarizer/.venv/bin/python3.11: No such file or directory
/Users/brunoferreira/Downloads/End-to-End MLOps/textsummarizer/.venv/bin/pip: line 2: exec: /Users/brunoferreira/Downloads/End-to-End MLOps Bootcamp/textsummarizer/.venv/bin/python3.11: cannot execute: No such file or directory
/Users/brunoferreira/Downloads/End-to-End MLOps/textsummarizer/.venv/bin/pip: line 2: /Users/brunoferreira/Downloads/End-to-End MLOps Bootcamp/textsummarizer/.venv/bin/python3.11: No such file or directory
/Users/brunoferreira/Downloads/End-to-End MLOps/textsummarizer/.venv/bin/pip: line 2: exec: /Users/brunoferreira/Downloads/End-to-End MLOps Bootcamp/textsummarizer/.venv/bin/python3.11: cannot execute: No such file or directory
/Users/brunoferreira/Downloads/End-to-End MLOps/textsummarizer/.venv/bin/pip: line 2: /Users/brunoferreira/Downloads/End-to-End MLOp

In [11]:
# Cell 1 — Setup (cwd + Hugging Face download config)
from pathlib import Path
import os

project_root = Path("/Users/brunoferreira/Downloads/End-to-End MLOps/textsummarizer")
os.chdir(project_root)

# Evita problemas de download em alguns ambientes
os.environ["HF_HUB_DISABLE_XET"] = "1"
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "0"

print("cwd:", Path.cwd())
print("config exists:", (project_root / "config" / "config.yaml").exists())
print("transformed dataset exists:", (project_root / "artifacts" / "data_transformation" / "samsum_dataset").exists())




cwd: /Users/brunoferreira/Downloads/End-to-End MLOps/textsummarizer
config exists: True
transformed dataset exists: True


In [12]:
# Cell 3 — Execução (forçando paths absolutos)
config = ConfigurationManager()
model_trainer_config = config.get_model_trainer_config()

model_trainer_config.data_path = project_root / "artifacts" / "data_transformation" / "samsum_dataset"
model_trainer_config.root_dir = project_root / "artifacts" / "model_trainer"

print("data_path:", model_trainer_config.data_path)
print("data_path exists:", model_trainer_config.data_path.exists())
print("root_dir:", model_trainer_config.root_dir)

model_trainer = ModelTrainer(config=model_trainer_config)
model_trainer.train()


[2026-02-22 17:48:44,149: INFO: common: yaml file: /Users/brunoferreira/Downloads/End-to-End MLOps/textsummarizer/config/config.yaml loaded successfully]
[2026-02-22 17:48:44,152: INFO: common: yaml file: /Users/brunoferreira/Downloads/End-to-End MLOps/textsummarizer/params.yaml loaded successfully]
[2026-02-22 17:48:44,152: INFO: common: created directory at: artifacts]
[2026-02-22 17:48:44,153: INFO: common: created directory at: artifacts/model_trainer]
data_path: /Users/brunoferreira/Downloads/End-to-End MLOps/textsummarizer/artifacts/data_transformation/samsum_dataset
data_path exists: True
root_dir: /Users/brunoferreira/Downloads/End-to-End MLOps/textsummarizer/artifacts/model_trainer
[2026-02-22 17:48:44,344: INFO: _client: HTTP Request: HEAD https://huggingface.co/google/pegasus-cnn_dailymail/resolve/main/config.json "HTTP/1.1 307 Temporary Redirect"]
[2026-02-22 17:48:44,359: INFO: _client: HTTP Request: HEAD https://huggingface.co/api/resolve-cache/models/google/pegasus-cnn_d

[2026-02-22 17:48:44,514: WARNING: _http: Warning: You are sending unauthenticated requests to the HF Hub. Please set a HF_TOKEN to enable higher rate limits and faster downloads.]
[2026-02-22 17:48:44,527: INFO: _client: HTTP Request: HEAD https://huggingface.co/api/resolve-cache/models/google/pegasus-cnn_dailymail/40d588fdab0cc077b80d950b300bf66ad3c75b92/tokenizer_config.json "HTTP/1.1 200 OK"]
[2026-02-22 17:48:44,542: INFO: _client: HTTP Request: GET https://huggingface.co/api/resolve-cache/models/google/pegasus-cnn_dailymail/40d588fdab0cc077b80d950b300bf66ad3c75b92/tokenizer_config.json "HTTP/1.1 200 OK"]
[2026-02-22 17:48:44,663: INFO: _client: HTTP Request: HEAD https://huggingface.co/google/pegasus-cnn_dailymail/resolve/main/tokenizer_config.json "HTTP/1.1 307 Temporary Redirect"]
[2026-02-22 17:48:44,678: INFO: _client: HTTP Request: HEAD https://huggingface.co/api/resolve-cache/models/google/pegasus-cnn_dailymail/40d588fdab0cc077b80d950b300bf66ad3c75b92/tokenizer_config.json 

Loading weights:   9%|▊         | 58/680 [00:00<00:00, 1276.30it/s, Materializing param=model.decoder.layers.2.encoder_attn.k_proj.weight]    

[2026-02-22 17:50:11,245: INFO: _client: HTTP Request: GET https://huggingface.co/api/models/google/pegasus-cnn_dailymail/xet-read-token/7db4e727842ad86cc5cdfeaef4d14c6fb2b46511 "HTTP/1.1 200 OK"]


Loading weights: 100%|██████████| 680/680 [00:00<00:00, 1343.03it/s, Materializing param=model.shared.weight]                                   
The tied weights mapping and config for this model specifies to tie model.shared.weight to model.decoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie model.shared.weight to model.encoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
PegasusForConditionalGeneration LOAD REPORT from: google/pegasus-cnn_dailymail
Key                                  | Status  | 
-------------------------------------+---------+-
model.encoder.embed_positions.weight | MISSING | 
model.decoder.embed_positions.weight | MISSING | 

Notes:
- MISSING	:t

[2026-02-22 17:50:13,372: INFO: _client: HTTP Request: HEAD https://huggingface.co/google/pegasus-cnn_dailymail/resolve/main/generation_config.json "HTTP/1.1 307 Temporary Redirect"]
[2026-02-22 17:50:13,436: INFO: _client: HTTP Request: HEAD https://huggingface.co/api/resolve-cache/models/google/pegasus-cnn_dailymail/40d588fdab0cc077b80d950b300bf66ad3c75b92/generation_config.json "HTTP/1.1 200 OK"]
[2026-02-22 17:50:13,492: INFO: _client: HTTP Request: GET https://huggingface.co/api/resolve-cache/models/google/pegasus-cnn_dailymail/40d588fdab0cc077b80d950b300bf66ad3c75b92/generation_config.json "HTTP/1.1 200 OK"]


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.
/Users/brunoferreira/Downloads/End-to-End MLOps/textsummarizer/.venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  super().__init__(loader)


RuntimeError: MPS backend out of memory (MPS allocated: 6.87 GiB, other allocations: 2.20 GiB, max allowed: 9.07 GiB). Tried to allocate 512.00 KiB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).